### 공기의 질을 측정한 데이터
1. airquanlity 데이터에 대해서 결측치가 가장 많은 변수를 찾아서 해당 결측치를 0으로 대체하고 결측치를 제외한 평균과 0으로 대치한 후 평균의 차이를 구하시오
2. wind 변수에 대해서 Min-Max 정규화를 수행 한 후 평균 값과 Z 정규화를 수행 한 후 평균 값의 차이를 구하시오
3. 월별 평균 기온을 구하시오

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('./csv_assignment/airquality.csv')
df.head()

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
4,NaN,NaN,14.3,56,5,5


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Ozone    116 non-null    float64
 1   Solar.R  146 non-null    float64
 2   Wind     153 non-null    float64
 3   Temp     153 non-null    int64  
 4   Month    153 non-null    int64  
 5   Day      153 non-null    int64  
dtypes: float64(3), int64(3)
memory usage: 7.3 KB


In [24]:
# 아래 셀 반복문 위한 확인용
print(len(df.index))
df['Ozone'].isna().value_counts()[1]


153


37

In [23]:
# 1. the most 결측치 -> 상단 셀의 info()로 추측 가능 
    # 0으로 대체
    # 결측제외 mean - 0대체후 mean 차이 

for i in df.columns :
    na_col = []
    na_num = df[i].isna().value_counts()[-1]
    
    # 결측치 없는 컬럼출력은 pass
    if na_num == len(df.index) :
        pass
    else:
        print(f'컬럼 {i} 결측치 수 : {na_num}')



컬럼 Ozone 결측치 수 : 37
컬럼 Solar.R 결측치 수 : 7


In [30]:
# 1-1. Ozone 컬럼이 결측치 더 많기에 Ozone 컬럼 결측치 0으로 대체 
df['Ozone_na_zero'] = df['Ozone'].fillna(0)
mean_0 = df['Ozone_na_zero'].mean()
print(f'0으로 대체시 Ozone 컬럼 평균 : {mean_0 : .4f}')

0으로 대체시 Ozone 컬럼 평균 :  31.9412


In [36]:
# 1-2. 확인용
    # 결측치 있을 때, 바로 mean() 하면 결측치갯수를 분모에서 제외하는지 확인용 
a = pd.DataFrame({'a':[1,2,3], 'b' : [4,5, np.nan]})
a['b'].mean()


4.5

In [38]:
# 1-2. 결측제외 mean
 
mean_o3 = df['Ozone'].mean()

print(f'결측치를 제외한 후 Ozone컬럼 평균 : {mean_o3:.4f}')



결측치를 제외한 후 Ozone컬럼 평균 : 42.1293


In [40]:
# 1-3. mean_0 과 mean_o3 차이

abs(mean_0 - mean_o3)

10.188133874239352

In [45]:
# 2. 정규화 수행후 평균 차이 

    # 2-1. minMax 스케일링  평균
from sklearn.preprocessing import MinMaxScaler

mM_scale = MinMaxScaler()

# 스케일링 진행과 반환 한번에하는 fit_transform을 Series형태로 불가하기에 np.array & reshape 적용 
    # reshape(-1,1) : 2차원 배열인데, 행의 수는 -1(임의 지정) & 열의 수는 1 로 차원 변환 
mM_mean = mM_scale.fit_transform(np.array(df['Wind']).reshape(-1, 1)).mean()

mM_mean

0.4346061231510147

In [46]:
df['Wind'] # minMaxScaler 적용 시, 본래의 df 값까지 바로 바뀌는지 확인용 -> 본래 df 불변

0       7.4
1       8.0
2      12.6
3      11.5
4      14.3
       ... 
148     6.9
149    13.2
150    14.3
151     8.0
152    11.5
Name: Wind, Length: 153, dtype: float64

In [48]:
    # 2-2. Z(표준정규화)

from sklearn.preprocessing import StandardScaler

z_scale = StandardScaler()

# fit_transform : 스케일링 진행 & 반환 동시에 
mean_z = z_scale.fit_transform(np.array(df['Wind']).reshape(-1,1)).mean()
mean_z

-2.322035084183334e-17

In [50]:
# 2-3. 평균 차이 

print(f'min-Max 정규화 후 평균과 Z 정규화 후 평균의 차이 : {abs(mM_mean - mean_z)}')

min-Max 정규화 후 평균과 Z 정규화 후 평균의 차이 : 0.4346061231510147


In [51]:
# 3. 사전 확인용
df['Month'].unique()

array([5, 6, 7, 8, 9], dtype=int64)

In [53]:
# 3. 사전 확인용
df[['Wind']].loc[df['Month']==5].mean()

Wind    11.622581
dtype: float64

In [58]:
# 3. 월별 평균 기온

for i in df['Month'].unique() :
    print(f'{i}월 평균기온 :', np.round(df[['Wind']].loc[df['Month'] == i ].mean()[0],4))

5월 평균기온 : 11.6226
6월 평균기온 : 10.2667
7월 평균기온 : 8.9419
8월 평균기온 : 8.7935
9월 평균기온 : 10.18


In [ ]:
# 3. 월별 평균 기온 - 다른 방법 - 간단ver

df.groupby('Month')['Temp'].mean() 

### 근무자 정보를 측정한 데이터
1. Attrition은 퇴사 여부 데이터를 (yes:1, no:0)으로 데이터를 변환하여 새로운 파생변수에 대입하고 범주별 레코드 수를 계산하시오
2. 범주형 변수 중 유일한 값을 1개만 가지고 있는 컬럼을 찾아내어 그 컬럼을 데이터에서 제거하시오
3. 원래 데이터에서 숫자형인 컬럼만 추출하여 새로운 데이터프레임으로 생성하고, 각 변수간의 상관계수(피어슨 상관계수)를 구하시오

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('./csv_assignment/HR-Employee-Attrition.csv')
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [6]:
# 1. Attrition : yes-> 1, no-> 0 인 new colounm & 범주별 레코드 수 확인 
df['Attrition'].value_counts()

No     1233
Yes     237
Name: Attrition, dtype: int64

In [8]:

df['YorN'] = df['Attrition'].replace({'Yes' : 1, 'No' : 0})

In [9]:
df['YorN'].value_counts() # Attrition 컬럼과 valuecount 동일 확인 

0    1233
1     237
Name: YorN, dtype: int64

In [17]:
# 2. 범주형 변수 중 유일한 값을 1개만 가지고 있는 컬럼을 찾아내어 그 컬럼을 데이터에서 제거
df['Over18'].value_counts()

Y    1470
Name: Over18, dtype: int64

In [21]:
df.drop('Over18', axis=1, inplace=True)

In [27]:
df.columns.isin(['YorN']).sum()  # 아래 Cell 위한 코드 확인용 

1

In [26]:
df.columns.isin(['Over18']).sum() # 'Over18'컬럼 삭제 확인 

0

In [28]:
# 3. 원래 데이터에서 숫자형인 컬럼만 추출하여 새로운 데이터프레임으로 생성하고, 각 변수간의 상관계수(피어슨 상관계수)


In [22]:
# dtypes : int64(27), object(8) 각 컬럼 유형 및 갯수 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [ ]:
# 빈 dataframe 생성
df_int = pd.DataFrame()


In [37]:
# 아래 셀 반복문을 위한 확인용
df['YorN'].dtype

int64


In [38]:
for i in df.columns :
    if df[i].dtype == 'int64' :
        df_int[i] = df[i]
    


In [39]:
df_int # df.info() 에서 확인한 int64 컬럼 수와 동일한지 확인 -> 27개로 동일 

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,YorN
0,41,1102,1,2,1,1,2,94,3,2,...,80,0,8,0,1,6,4,0,5,1
1,49,279,8,1,1,2,3,61,2,2,...,80,1,10,3,3,10,7,1,7,0
2,37,1373,2,2,1,4,4,92,2,1,...,80,0,7,3,3,0,0,0,0,1
3,33,1392,3,4,1,5,4,56,3,1,...,80,0,8,3,3,8,7,3,0,0
4,27,591,2,1,1,7,1,40,3,1,...,80,1,6,3,3,2,2,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,884,23,2,1,2061,3,41,4,2,...,80,1,17,3,3,5,2,0,3,0
1466,39,613,6,1,1,2062,4,42,2,3,...,80,1,9,5,3,7,7,1,7,0
1467,27,155,4,3,1,2064,2,87,4,2,...,80,1,6,0,3,6,2,0,3,0
1468,49,1023,2,3,1,2065,4,63,2,2,...,80,0,17,3,2,9,6,0,8,0


In [40]:
# Pearson correlation coefficient = cov(x,y) / std_x * std_y

df_int.corr()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,YorN
Age,1.000000,0.010661,-0.001686,0.208034,NaN,-0.010145,0.010146,0.024287,0.029820,0.509604,...,NaN,0.037510,0.680381,-0.019621,-0.021490,0.311309,0.212901,0.216513,0.202089,-0.159205
DailyRate,0.010661,1.000000,-0.004985,-0.016806,NaN,-0.050990,0.018355,0.023381,0.046135,0.002966,...,NaN,0.042143,0.014515,0.002453,-0.037848,-0.034055,0.009932,-0.033229,-0.026363,-0.056652
DistanceFromHome,-0.001686,-0.004985,1.000000,0.021042,NaN,0.032916,-0.016075,0.031131,0.008783,0.005303,...,NaN,0.044872,0.004628,-0.036942,-0.026556,0.009508,0.018845,0.010029,0.014406,0.077924
Education,0.208034,-0.016806,0.021042,1.000000,NaN,0.042070,-0.027128,0.016775,0.042438,0.101589,...,NaN,0.018422,0.148280,-0.025100,0.009819,0.069114,0.060236,0.054254,0.069065,-0.031373
EmployeeCount,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EmployeeNumber,-0.010145,-0.050990,0.032916,0.042070,NaN,1.000000,0.017621,0.035179,-0.006888,-0.018519,...,NaN,0.062227,-0.014365,0.023603,0.010309,-0.011240,-0.008416,-0.009019,-0.009197,-0.010577
EnvironmentSatisfaction,0.010146,0.018355,-0.016075,-0.027128,NaN,0.017621,1.000000,-0.049857,-0.008278,0.001212,...,NaN,0.003432,-0.002693,-0.019359,0.027627,0.001458,0.018007,0.016194,-0.004999,-0.103369
HourlyRate,0.024287,0.023381,0.031131,0.016775,NaN,0.035179,-0.049857,1.000000,0.042861,-0.027853,...,NaN,0.050263,-0.002334,-0.008548,-0.004607,-0.019582,-0.024106,-0.026716,-0.020123,-0.006846
JobInvolvement,0.029820,0.046135,0.008783,0.042438,NaN,-0.006888,-0.008278,0.042861,1.000000,-0.012630,...,NaN,0.021523,-0.005533,-0.015338,-0.014617,-0.021355,0.008717,-0.024184,0.025976,-0.130016
JobLevel,0.509604,0.002966,0.005303,0.101589,NaN,-0.018519,0.001212,-0.027853,-0.012630,1.000000,...,NaN,0.013984,0.782208,-0.018191,0.037818,0.534739,0.389447,0.353885,0.375281,-0.169105
